In [26]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy.linalg import logm, expm

dim = 4

sigma_z = np.array([[1, 0],[0, -1]])
sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0, -1j], [1j, 0]])
identity = np.identity(2)
sigmas = [identity, sigma_x, sigma_y, sigma_z]
sigmas_4d = np.kron(sigmas, sigmas)

# epsilon=0.0000001
# oepsilon=1-epsilon

def random_hamiltonian_and_rho(beta):
    a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    H = (a.conj().T + a)/2
    eigenval,eigenvecs=np.linalg.eig(H)
    eigenval=np.e**(-beta*eigenval)
    eigenval=np.diag(eigenval)
    s=eigenval.dot(eigenvecs.conj().T)
    rho=eigenvecs.dot(s)
    rho=rho/np.trace(rho)
    return H, rho

def pure(beta):
    H=np.random.uniform(-1,1,size=dim)
    rho=np.e**(-beta*H)
    H=np.diag(H)
    rho=np.diag(rho)
    rho/=np.trace(rho)
    return H,rho

def matrix_to_free_parameters(matr):
    free_parameters = np.real(np.array([np.trace(matr.dot( measurement)) for measurement in sigmas_4d]))
    return free_parameters


def free_parameters_to_matrix(free_parameters):
    matr = np.zeros((4,4))+0j*np.zeros((4,4))
    for i in range(dim**2):
        matr += free_parameters[i]*sigmas_4d[i]

    return matr/4

# This function is new and generates non-equilibrium states and calculates the temperature according to the paper
def nongibbs():
    a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    H = (a.conj().T + a)/2
    a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    rho=a.dot(a.conj().T)
    rho/=np.trace(rho)
    t=np.trace(H)
    h=np.sqrt(np.trace(H.dot(H)-t**2/dim))
    o1=(H-t/dim)/h
    beta=np.trace(o1.dot(np.log(rho)))/(-h)
    return H,rho,beta
# print(nongibbs())
# a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
# H = (a.conj().T + a)/2
# print(H - free_parameters_to_matrix(matrix_to_free_parameters(H)))


# hamiltonian , rho = hermitian(3, 0.5)
# print(rho - sp.linalg.expm(-0.5*hamiltonian)/np.trace(sp.linalg.expm(-0.5*hamiltonian)))


# a = np.array([1,2,3])
# print(np.outer(a, a.conj()))

def cov(A, B):
    d = 4

    out =  np.trace(np.matmul(A,B))/d - np.trace(A)*np.trace(B) / d**2

    return out

def var_matrix(A):
    d = 4
    
    out = np.trace(np.matmul(A,A))/d - (np.trace(A)/d)**2

    return out

def non_thermal(H, rho):
    # a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    # H = (a.conj().T + a)/2
    # a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    # rho=a.dot(a.conj().T)
    # rho/=np.trace(rho)

    beta = np.real(cov(H, -logm(rho)) / var_matrix(H) )

    return H, rho, beta


def logarithm(H):
    eigenval,eigenvecs=np.linalg.eig(H)
    eigenval=np.log(eigenval)
    eigenval=np.diag(eigenval)
    s=eigenval.dot(eigenvecs.conj().T)
    loga=eigenvecs.dot(s)
    return loga

df = pd.read_csv('QT - dimension 4.csv')

H, rho = random_hamiltonian_and_rho(3)

print(non_thermal(H, rho)[-1])

for i in range(1, 100):
    H, rho = random_hamiltonian_and_rho(i)
    print(non_thermal(H, rho)[-1])



3.0000000000001745
0.9999999999999998
1.9999999999999991
2.999999999999918
3.9999999999978044
5.000000000005336
5.9999999999712825
6.999999999676897
7.999999848775509
9.00000008770419
9.999999974676241
11.000437655676082
11.997598093423106
13.000382166990393
11.900065102939681
13.347798691330564
13.249029076258454
13.235726317885016
11.693618367346103
15.016974718628111
13.06078817640765
16.185687466638473
12.161005370615454
10.125164310952037
13.26953020067974
14.064780393968535
8.232625360053568
12.584899653154237
15.020650077661347
18.04615452318767
9.797816391111814
16.80101208039641
13.126732948970183
18.941953558002396
17.98506788503345
11.696434602517641
16.328321190904454
12.781546678249263
13.657061002591194
4.852829142420365
11.823446070743767
13.354650666736898
13.60835256149562
11.704066381635927
11.740429830424034
12.642946551435205
19.198905943030812
10.850912514131673
2.470906548629354
13.117987016178825
6.757835458758059
2.4746839666143368
5.008860989330671
5.2162779938

In [4]:
np.array([1,2]).shape

(2,)

In [5]:
numdata = 10**5




cols = []

for i in range(1, 17):
    cols.append('fp hamiltonian'+str(i))

for i in range(1, 17):
    cols.append('fp rho'+str(i))

cols.append('beta')

# H=np.zeros((numdata,dim,dim),dtype=np.complex_)
# rho=np.zeros((numdata,dim,dim),dtype=np.complex_)
beta = np.random.uniform(0, 1, numdata)

hamiltonian, rho = random_hamiltonian_and_rho(beta[0])

free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
free_params_rho = matrix_to_free_parameters(rho)

data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(beta[0])).reshape(1, 33)


# print(len(data), data)

df = pd.DataFrame(data, columns=cols)
print(df)
for i in range(1,numdata//10):
    hamiltonian,rho=pure(beta[i])
    free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
    free_params_rho = matrix_to_free_parameters(rho)
    data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(beta[i]))
    df.loc[len(df)] = data
for i in range(numdata//10,numdata):
    # print(i)
    hamiltonian, rho = random_hamiltonian_and_rho(beta[i])

    free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
    free_params_rho = matrix_to_free_parameters(rho)

    data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(beta[i]))
    df.loc[len(df)] = data





# for i in range(numdata):
#     H[i],rho[i]=hermitian(dim,beta[i])
# np.savez_compressed('Hamiltonians',H)
# np.savez_compressed('density matrices',rho)

   fp hamiltonian1  fp hamiltonian2  fp hamiltonian3  fp hamiltonian4  \
0         1.815262         0.063787         -0.98291         1.123441   

   fp hamiltonian5  fp hamiltonian6  fp hamiltonian7  fp hamiltonian8  \
0        -0.661134         1.925564         0.329642         -0.55569   

   fp hamiltonian9  fp hamiltonian10  ...   fp rho8   fp rho9  fp rho10  \
0        -0.408958          1.564257  ...  0.051871  0.056905 -0.161987   

   fp rho11  fp rho12  fp rho13  fp rho14  fp rho15  fp rho16      beta  
0  0.152104  0.039256  0.073198  0.019943  0.003156 -0.050396  0.438644  

[1 rows x 33 columns]


In [42]:
cols = []

for i in range(1, 17):
    cols.append('fp hamiltonian'+str(i))

for i in range(1, 17):
    cols.append('fp rho'+str(i))

cols.append('beta')
numdata=2*10**4
hamiltonian, rho, betas = nongibbs()
free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
free_params_rho = matrix_to_free_parameters(rho)
print(betas,betas.real)
data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(betas.real)).reshape(1, 33)
df = pd.DataFrame(data, columns=cols)
for i in range(1,numdata):
    hamiltonian, rho,betas = nongibbs()
    free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
    free_params_rho = matrix_to_free_parameters(rho)
    data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(betas.real))
    df.loc[len(df)] = data
df.to_csv('nonthermal - dimension 4.csv')

(-1.384321226799688-0j) -1.384321226799688


In [24]:
def cov(A, B):
    d = 4

    out =  np.trace(np.matmul(A,B))/d - np.trace(A)*np.trace(B) / d**2

    return out

def var_matrix(A):
    d = 4
    
    out = np.trace(np.matmul(A,A))/d - (np.trace(A)/d)**2

    return out

def non_thermal():
    a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    H = (a.conj().T + a)/2
    a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    rho=a.dot(a.conj().T)
    rho/=np.trace(rho)

    beta = np.real(cov(H, -logm(rho)) / var_matrix(H) )

    return H, rho, beta



cols = []

for i in range(1, 17):
    cols.append('fp hamiltonian'+str(i))

for i in range(1, 17):
    cols.append('fp rho'+str(i))

cols.append('beta')
numdata=10**4
hamiltonian, rho, beta = non_thermal()
free_params_hamiltonian = np.real(matrix_to_free_parameters(hamiltonian))
free_params_rho = np.real(matrix_to_free_parameters(rho))
# print(betas,betas.real)
data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(np.real(beta))).reshape(1, 33)
df = pd.DataFrame(data, columns=cols)
for i in range(1,numdata):
    if i %1000 ==0:
        print(i)
    hamiltonian, rho, beta = nongibbs()
    if beta>1:
        continue
    free_params_hamiltonian = np.real(matrix_to_free_parameters(hamiltonian))
    free_params_rho = np.real(matrix_to_free_parameters(rho))
    data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(np.real(beta)))
    df.loc[len(df)] = data
df.to_csv('non__thermal - dimension 4_final.csv')

1000
2000
3000
4000
5000
6000
7000
8000
9000


In [25]:
len(df)

6658